1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)

In [113]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
from pymongo import MongoClient

In [114]:
User_Agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

In [115]:
def parsing_hhru_json(User_Agent, url):
    headers = {'User-Agent': User_Agent,}
    vacancy_from_hh = []
    n = 0
    response = requests.get(url=url.format(n), headers=headers)
    while response.status_code != 404:
        response = requests.get(url=url.format(n), headers=headers)
        soup = bs(response.content, 'html.parser')
        list_vacancy = soup.find_all('a', {'class': "serp-item__title"})
        for i in range(len(list_vacancy)-1):
            name_vacancy = (list_vacancy[i].getText())
            link_vacancy = (list_vacancy[i]['href'])
            salary_data = list_vacancy[i].parent.parent.parent.find_all('span', {'data-qa': "vacancy-serp__vacancy-compensation"})
            if len(salary_data) == 0:
                min_salary = None
                max_salary = None
                currency = None
            else:
                salary_data = salary_data[0].get_text().replace('\u202f', '').split()

                if len(salary_data) == 3 and salary_data[0] == 'от':
                    min_salary = int(salary_data[1])
                    max_salary = None
                    currency = salary_data[2]

                if len(salary_data) == 3 and salary_data[0] == 'до':
                    min_salary = None
                    max_salary = int(salary_data[1])
                    currency = salary_data[2]

                if len(salary_data) == 4:
                    min_salary = int(salary_data[0])
                    max_salary = int(salary_data[2])
                    currency = salary_data[3]
            vacancy_from_hh.append({
                'name_vacancy': name_vacancy,
                'link_vacancy':link_vacancy, 
                'min_salary' : min_salary,
                'max_salary' : max_salary, 
                'currency' :currency, 
                'source' : 'https://hh.ru/'      
            })
        n = n+1
        sleep(5)       
    return vacancy_from_hh

In [116]:
url = 'https://hh.ru/search/vacancy?text=python&from=suggest_post&area=1&customDomain=1&page={}&hhtmFrom=vacancy_search_list'
vacancy_from_hh = parsing_hhru_json(User_Agent = User_Agent, url = url)
vacancy_from_hh

[{'name_vacancy': 'Senior системный разработчик (python, Linux)',
  'link_vacancy': 'https://hh.ru/vacancy/73442946?from=vacancy_search_list&query=python',
  'min_salary': None,
  'max_salary': None,
  'currency': None,
  'source': 'https://hh.ru/'},
 {'name_vacancy': 'Junior Python Developer/Analyst (remote)',
  'link_vacancy': 'https://hh.ru/vacancy/74824002?from=vacancy_search_list&query=python',
  'min_salary': 800,
  'max_salary': None,
  'currency': 'USD',
  'source': 'https://hh.ru/'},
 {'name_vacancy': 'Разработчик C# / Python',
  'link_vacancy': 'https://hh.ru/vacancy/71147348?from=vacancy_search_list&query=python',
  'min_salary': 300000,
  'max_salary': 500000,
  'currency': 'руб.',
  'source': 'https://hh.ru/'},
 {'name_vacancy': 'Python Backend разработчик',
  'link_vacancy': 'https://hh.ru/vacancy/73581287?from=vacancy_search_list&query=python',
  'min_salary': 130000,
  'max_salary': 200000,
  'currency': 'руб.',
  'source': 'https://hh.ru/'},
 {'name_vacancy': 'Разработ

In [117]:
client = MongoClient()

In [118]:
client.list_database_names()

['admin', 'config', 'local', 'vacancies_python']

In [125]:
db = client.vacancies_python
vacancies_hh_ru = db.python_hh_ru
for vacancy in vacancy_from_hh:
    vacancies_hh_ru.insert_one(vacancy)

In [127]:
def add_new_vacancy(collections):
    vacancy_from_hh = parsing_hhru_json(User_Agent = User_Agent, url = url)
    for vacancy in vacancy_from_hh:
        if len(list(collections.find({'link_vacancy': vacancy['link_vacancy']}))) == 0:
            collections.insert_one(vacancy) 

In [130]:
add_new_vacancy(collections = vacancies_hh_ru)

In [132]:
def seach_salary_vacancy(salary,currency):
    rub_vacancy = db.rub_vacancy
    for vacancy in vacancies_hh_ru.find({'currency' : 'руб.'}):
        rub_vacancy.insert_one(vacancy)
    USD_vacancy = db.USD_vacancy
    for vacancy in vacancies_hh_ru.find({'currency' : 'USD'}):
        USD_vacancy.insert_one(vacancy)
    if currency == 'руб.':
        pprint(list(rub_vacancy.find({'$or': [{'min_salary' :{'$gte': salary}}, {'max_salary' : {'$gte': salary}}]})))
        salary = salary/70
        pprint(list(USD_vacancy.find({'$or': [{'min_salary' :{'$gte': salary}}, {'max_salary' : {'$gte': salary}}]})))            
    if currency == 'USD':
        pprint(list(USD_vacancy.find({'$or': [{'min_salary' :{'$gte': salary}}, {'max_salary' : {'$gte': salary}}]})))
        salary = salary/70
        pprint(list(rub_vacancy.find({'$or': [{'min_salary' :{'$gte': salary}}, {'max_salary' : {'$gte': salary}}]})))
    db.rub_vacancy.drop()
    db.USD_vacancy.drop()

In [133]:
salary = 300000
currency = 'руб.'
seach_salary_vacancy(salary,currency)

[{'_id': ObjectId('63b4541530d931ddb4c88693'),
  'currency': 'руб.',
  'link_vacancy': 'https://hh.ru/vacancy/71147348?from=vacancy_search_list&query=python',
  'max_salary': 500000,
  'min_salary': 300000,
  'name_vacancy': 'Разработчик C# / Python',
  'source': 'https://hh.ru/'},
 {'_id': ObjectId('63b4541530d931ddb4c886a5'),
  'currency': 'руб.',
  'link_vacancy': 'https://hh.ru/vacancy/73256225?from=vacancy_search_list&query=python',
  'max_salary': 350000,
  'min_salary': 250000,
  'name_vacancy': 'Ведущий full-stack разработчик (Python, JS)',
  'source': 'https://hh.ru/'},
 {'_id': ObjectId('63b4541530d931ddb4c886cb'),
  'currency': 'руб.',
  'link_vacancy': 'https://hh.ru/vacancy/69537461?from=vacancy_search_list&query=python',
  'max_salary': 400000,
  'min_salary': 200000,
  'name_vacancy': 'Senior Python разработчик (Intelligent Document Processing)',
  'source': 'https://hh.ru/'},
 {'_id': ObjectId('63b4541530d931ddb4c886d4'),
  'currency': 'руб.',
  'link_vacancy': 'https:/